# Web pages analysis Q&A with Azure Open AI and Azure Cognitive Search

In [1]:
import datetime
import openai
import os
import requests
import sys

from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.vectorstores import AzureSearch

In [2]:
sys.version

'3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 23-Oct-2023 09:35:30


In [4]:
print("Open AI version:", openai.__version__)

Open AI version: 0.28.1


## Web pages analysis

In [5]:
urls = [
    "https://learn.microsoft.com/en-us/azure/ai-services/openai/faq",
    "https://learn.microsoft.com/en-us/azure/ai-services/openai/overview",
    "https://learn.microsoft.com/en-us/azure/ai-services/openai/quotas-limits",
    "https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models",
    "https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/legacy-models",
    "https://learn.microsoft.com/en-us/azure/ai-services/openai/whats-new",
]

In [6]:
DATA_DIR = "txt"

os.makedirs(DATA_DIR, exist_ok=True)

In [7]:
for i in range(len(urls)):
    response = requests.get(urls[i])

    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")
        # Extract all the text content from the HTML
        text = soup.get_text()
        text = text.replace("\n", "").replace("\r", "").replace("\t", "")
        # Saving the extracted text as a text file
        text_file = os.path.join(DATA_DIR, urls[i].split("/")[-1] + ".txt")
        with open(text_file, "w") as file:
            file.write(text)
    else:
        print(f"Failed to retrieve the web page. Status code: {response.status_code}")

In [8]:
!ls $DATA_DIR/*.txt -lh

-rwxrwxrwx 1 root root  41K Oct 23 09:35 txt/faq.txt
-rwxrwxrwx 1 root root 6.4K Oct 23 09:35 txt/legacy-models.txt
-rwxrwxrwx 1 root root 9.4K Oct 23 09:35 txt/models.txt
-rwxrwxrwx 1 root root 7.5K Oct 23 09:35 txt/overview.txt
-rwxrwxrwx 1 root root 5.3K Oct 23 09:35 txt/quotas-limits.txt
-rwxrwxrwx 1 root root  13K Oct 23 09:35 txt/whats-new.txt


In [9]:
docs = []

for file in [f for f in os.listdir(DATA_DIR) if f.endswith(".txt")]:
    print("Processing file {file}")
    try:
        loader = TextLoader(os.path.join(DATA_DIR, file), encoding="utf-8")
        docs.extend(loader.load_and_split())
    except Exception as e:
        pass

Processing file {file}
Processing file {file}
Processing file {file}
Processing file {file}
Processing file {file}
Processing file {file}


In [10]:
len(docs)

24

In [11]:
index_name = "web-crawlerdemo"

In [12]:
load_dotenv("azure.env")

True

## Azure Cognitive Search

In [13]:
# Initialize our embedding model
embeddings = OpenAIEmbeddings(
    deployment=os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME"),
    model=os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME"),
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    openai_api_type="azure",
    chunk_size=1,
)

In [14]:
# Set our Azure Search
acs = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_COGNITIVE_SEARCH_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY"),
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [15]:
# Add documents to Azure Search
acs.add_documents(documents=docs)

['NDcyMDFlZmEtZmZjZC00YWU4LWFjNDItOWJiNzMwOGI1ZTM1',
 'MzIwNGQyMWItNjRmZS00MWNkLTg3ZTYtNDIyZmE4NTJhNjJj',
 'NDUxZmRiM2EtMWI1Ni00YzQ4LWJhN2EtMTViOTExNTUwZGQy',
 'NThiMzcxYTAtMWYxOS00ZWNmLWJiMDYtM2FjZDU2YjA3YWIy',
 'N2ZiODRjN2ItNmZmNy00MTExLWIyMTQtYzNlYTY5ZDg4NWI1',
 'ZDI4YzhhZWEtYWYyYS00NjljLThlNjEtMTQ0OGZhNGEyYTBi',
 'NDA3ODczMTQtODg2OC00ZTQ1LTg1MmMtZWY0NDZjNzM2ZmI5',
 'MzA0YmYzYjEtNDQxNi00NWY4LWE1NDAtOTQ1NTZmYmZhNzFj',
 'NDkyMDFiZWYtZjk5ZC00MDFkLWFkYjUtZDUzYWVjMTkyMDRj',
 'ZWQ5MzFmMTYtZTlhNi00M2Q2LWE3NTYtNmU5MzM0YzkyNDA2',
 'YjRiN2Q4OGUtODY0OS00OWJjLTg5YmEtOWE3MjJmMDNlNWY5',
 'YTZiYWUwYzctYmNiOC00ZDdkLTljZWEtNTJlZmJiMjNmNmY0',
 'NjZiZTNiYmYtYTRkMS00MjhmLWJhOTgtZjQxODg1MDRkZTU4',
 'MmEyNDIyNWUtNDZlMC00Y2FkLTg4NzUtY2VhNWVmNTdjYmU2',
 'N2M0OWY5YmYtYmY5Zi00YWMyLThkNWYtMmEyMzI4Zjg2ODZj',
 'OTViZDUyMzctZGRmNC00Nzg2LThmYzktM2M1N2RjNjUzYzNm',
 'ODZkY2JmMmEtMDEwZC00MjY4LWI1ZjYtYTFmMTlkNzcyYzky',
 'NGEwODczMzctZGI5YS00NzUxLTkyZTgtMjI1ZjBiOWJhMjdk',
 'NDBjZDAwYmMtYjFiNi00MGI4LThiN2MtMmExYTg1MDI3

In [16]:
# Define Azure Cognitive Search as our retriever
retriever = AzureCognitiveSearchRetriever(
    content_key="content", top_k=5, index_name=index_name
)

In [17]:
# Set chatGPT 3.5 as our LLM
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo-16k", temperature=0.2)

## Testing

In [18]:
# Define a template message
template = """
You are an AI expert powered by Azure Open AI.
You are going to analyse some text documents about Azure Open AI.
If you do not know just say you do not know.
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Set the Retrieval QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    return_source_documents=True,
)

In [19]:
def qa():
    """
    Get answer
    """
    result = qa_chain({"query": question, "chat_history": chat_history})
    chat_history.append((question, result))
    source = result["source_documents"][0].metadata["metadata"]

    print(f"Question: {question} \n")
    print(f"Answer: {result['result']} \n")
    print(f"{source}")

In [20]:
chat_history = []

In [21]:
question = "Who are you?"

res = qa()

Question: Who are you? 

Answer: I am an AI expert powered by Azure Open AI. I am here to provide information and answer questions about Azure Open AI. Thanks for asking! 

{"source": "txt/faq.txt"}


In [22]:
question = "Is it possible to use GPT4?"

qa()

Question: Is it possible to use GPT4? 

Answer: Yes, Azure OpenAI supports the latest GPT-4 models. It supports both GPT-4 and GPT-4-32K. Thanks for asking! 

{"source": "txt/faq.txt"}


In [23]:
question = "How about GPT5?"

qa()

Question: How about GPT5? 

Answer: I'm sorry, but I do not have information about GPT-5. Thanks for asking! 

{"source": "txt/faq.txt"}


In [24]:
question = "How about the quotas?"

qa()

Question: How about the quotas? 

Answer: The quotas and limits for Azure OpenAI are subject to change, but here are some of the default quotas and limits:

- OpenAI resources per region per Azure subscription: 30
- Default DALL-E quota limits: 2 concurrent requests
- Maximum prompt tokens per request: Varies per model
- Max fine-tuned model deployments: 2
- Total number of training jobs per resource: 100
- Max simultaneous running training jobs per resource: 1
- Max training jobs queued: 20
- Max Files per resource: 30
- Total size of all files per resource: 1 GB
- Max training job time: 720 hours
- Max training job size: 2 Billion tokens in training file
- Max size of all files per upload (Azure OpenAI on your data): 16 MB

Please note that these are just some of the quotas and limits, and there may be additional limits depending on the specific model and region. It's always a good idea to check the official documentation for the most up-to-date information on quotas and limits.

Tha

In [25]:
question = "What is the tokens per minutes quota for gpt-35-turbo?"

qa()

Question: What is the tokens per minutes quota for gpt-35-turbo? 

Answer: The tokens per minute quota for gpt-35-turbo varies depending on the region. In regions such as East US, South Central US, West Europe, France Central, UK South, the quota is 240K tokens per minute. In regions like North Central US, Australia East, East US 2, Canada East, Japan East, Sweden Central, Switzerland North, the quota is 300K tokens per minute. Thanks for asking! 

{"source": "txt/whats-new.txt"}


In [26]:
question = "How to do quota increase request?"

qa()

Question: How to do quota increase request? 

Answer: To request a quota increase, you can submit a request from the Quotas page of Azure OpenAI Studio. However, please note that due to overwhelming demand, quota increase requests are being accepted and filled in the order they are received. Priority is given to customers who generate traffic that consumes the existing quota allocation, and your request may be denied if this condition is not met. For other rate limits, you can submit a service request. Thanks for asking! 

{"source": "txt/quotas-limits.txt"}


In [27]:
question = "What are the names of the embeddings models?"

qa()

Question: What are the names of the embeddings models? 

Answer: The names of the embeddings models are:

1. text-embedding-ada-002 (Version 2)
2. DALL-E (Preview)
3. Whisper (Preview)

Thanks for asking! 

{"source": "txt/faq.txt"}


In [28]:
question = "How to update an index?"

qa()

Question: How to update an index? 

Answer: You can update an index in Azure OpenAI on your data by scheduling an automatic index refresh or by uploading additional data to your Azure Blob Container and using it as your data source when you create a new index. The new index will include all of the data in your container. Thanks for asking! 

{"source": "txt/faq.txt"}


In [29]:
question = "What are the legacy models?"

qa()

Question: What are the legacy models? 

Answer: The legacy models in Azure OpenAI Service are GPT-3.5 and GPT-3. These models are no longer available for new deployments starting from July 6, 2023. However, deployments created prior to that date will remain available until July 5, 2024. It is recommended to migrate to the replacement models, such as GPT-3.5 Turbo Instruct, before the retirement date. 

{"source": "txt/faq.txt"}


In [30]:
question = "Can you tell me about GPT-3?"

qa()

Question: Can you tell me about GPT-3? 

Answer: Azure OpenAI Service offers GPT-3, which is a powerful language model developed by OpenAI. GPT-3 stands for "Generative Pre-trained Transformer 3" and it is designed to understand and generate natural language. It can be used for a wide range of applications such as text completion, language translation, question answering, and more. GPT-3 has been widely recognized for its ability to generate human-like text and has been used in various industries for different use cases. It is one of the models available in Azure OpenAI Service that customers can leverage for their language processing needs. Thanks for asking! 

{"source": "txt/faq.txt"}


In [31]:
question = "What are the updates for Canada East region?"

qa()

Question: What are the updates for Canada East region? 

Answer: The updates for the Canada East region in Azure OpenAI Service include:

- Azure OpenAI is now available in the Canada East region.
- Check the models page for the latest information on model availability in each region.

Thanks for asking! 

{"source": "txt/quotas-limits.txt"}


In [1]:
#chat_history